# Day 8 Solution

---

## Setup

In [1]:
import numpy as np

In [2]:
INPUT_FILE = 'input.txt'

In [4]:
with open(INPUT_FILE, 'r') as f:
    lines = [line.strip('\n') for line in f.readlines()]
    
# lines

---

## Create Tree Grid from Lines

In [5]:
def create_tree_grid(lines: list[str]) -> list[list[int]]:
    """Create an array of integers from a list of numeric strings."""
    rows = [[int(num) for num
             in list(line)]
            for line in lines]
    # Convert to numpy array so object can be transposed later
    rows = np.array(rows)
    
    return rows

---

## Create Checked Grid (Empty Grid of Zeros)

In [7]:
def create_empty_checked_grid():
    """Create an array with the same shape as tree_grid, filled with 0s"""
    rows, columns = tree_grid.shape
    
    empty_checked_grid = np.zeros((rows, columns),
                                  dtype=np.int16)
    
    return empty_checked_grid

---

### Example Tree Grid

In [9]:
# Example input, same format as "lines"

example_lines = [
    '30373',
    '25512',
    '65332',
    '33549',
    '35390'
]

Empty Grid

In [45]:
empty_checked_grid = create_empty_checked_grid()
empty_checked_grid

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int16)

---

Row View - Left to Right (Original)

In [62]:
tree_grid_original = create_tree_grid(example_lines)
# Or 
tree_grid_left_to_right = tree_grid_original
tree_grid_left_to_right

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 3, 3, 2],
       [3, 3, 5, 4, 9],
       [3, 5, 3, 9, 0]])

---

## EXAMPLES (ERASE)

In [109]:
tree_grid_example_left_to_right = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

tree_grid_example_left_to_right

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [110]:
tree_grid_example_right_to_left = np.fliplr(tree_grid_example_left_to_right)
tree_grid_example_right_to_left

array([[3, 2, 1],
       [6, 5, 4],
       [9, 8, 7]])

In [111]:
tree_grid_example_up_to_down = np.rot90(tree_grid_example_left_to_right, 3)
tree_grid_example_up_to_down = np.fliplr(tree_grid_example_up_to_down)
tree_grid_example_up_to_down

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

In [112]:
tree_grid_example_down_to_up = np.rot90(tree_grid_example_left_to_right, 3)
tree_grid_example_down_to_up

array([[7, 4, 1],
       [8, 5, 2],
       [9, 6, 3]])

---

Row View - Right to Left (Reversed)

In [96]:
reversed_tree_grid = np.fliplr(tree_grid)
# Or
tree_grid_right_to_left = reversed_tree_grid
tree_grid_right_to_left

array([[3, 7, 3, 0, 3],
       [2, 1, 5, 5, 2],
       [2, 3, 3, 5, 6],
       [9, 4, 5, 3, 3],
       [0, 9, 3, 5, 3]])

---

Column View - Up to Down

In [116]:
tree_grid_up_to_down = np.rot90(tree_grid_original, 3)
tree_grid_up_to_down = np.fliplr(tree_grid_up_to_down)
tree_grid_up_to_down

# To flip back to normal:

# tree_grid_back_to_normal = np.fliplr(tree_grid_up_to_down)
# tree_grid_back_to_normal = np.rot90(tree_grid_back_to_normal, 1)
# tree_grid_back_to_normal

array([[3, 2, 6, 3, 3],
       [0, 5, 5, 3, 5],
       [3, 5, 3, 5, 3],
       [7, 1, 3, 4, 9],
       [3, 2, 2, 9, 0]])

---

Column View - Down to Up

In [114]:
tree_grid_down_to_up = np.rot90(tree_grid_original, 3)
# tree_grid_down_to_up = np.fliplr(tree_grid_down_to_up)
# tree_grid_down_to_up

tree_grid_back_to_normal = np.rot90(tree_grid_down_to_up, 1)
tree_grid_back_to_normal

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 3, 3, 2],
       [3, 3, 5, 4, 9],
       [3, 5, 3, 9, 0]])

---

## Directional Checks 

### Edge Checks

In [17]:
def add_edge_trees_to_checked_grid(tree_grid):
    row_last_index = tree_grid.shape[0] - 1
    
    checked_grid_edges = create_empty_checked_grid()
    
    for i, row in enumerate(tree_grid):
        for j, tree in enumerate(row):
            if  (
                    # Tree is in first row or tree is in first column
                    (i == 0) or (j == 0) or
                    # Tree is in last row
                    (i == row_last_index) or
                    # Tree is in last column
                    (j == row_last_index)
            ):
                # Add a 1 to the edge tree's position
                checked_grid_edges[i][j] = 1
    
    return checked_grid_edges

---

### Row Checks

#### Left-to-Right

In [33]:
def check_row_left_to_right(tree_grid_left_to_right):
    """Create a grid of trees visible for each row when viewed from left to right."""
    
    # Initialize empty grid to add visible trees (1s) to
    checked_grid_left_to_right = create_empty_checked_grid()
    
    # For each row in the tree grid
    for i, row in enumerate(tree_grid_left_to_right):
        # Find the tallest tree for that row
        tallest_tree_height = max(row)
        
        for j, tree in enumerate(row):
            # For the first tree in the row, set it as the current tallest tree
            if j == 0:
                current_tree = row[j]
                checked_grid_left_to_right[i][j] = 1
                # If the first tree is the tallest tree, we don't need to check the rest of
                # the trees in the row, we can skip to checking the next row
                if current_tree == tallest_tree_height:
                    break
            
            # If the tree is not the first tree in the row, and the tree is the same height
            # as the tallest tree
            elif tree == tallest_tree_height:
                # Add a visible tree marker (1) to the checked_trees grid
                checked_grid_left_to_right[i][j] = 1
                # Then skip to the next row
                break
            
            # If the tree is not the first in the row, and it is not the tallest tree
            elif tree != tallest_tree_height:
                # If the tree's height is taller than the current tallest tree
                if tree > current_tree:
                    # Add a visible tree marker (1) to the checked_trees grid
                    checked_grid_left_to_right[i][j] = 1
                    # Then, change the current_tree to that tree, so that each following tree
                    # must be taller than it to be considered visible.
                    current_tree = tree
                    
                # If the tree is not the first in the row, and the tree is not taller than the
                # current tallest tree, it is not visible, so continue checking the trees in
                # the current row until the tallest tree is found.
                elif tree <= current_tree:
                    continue
                
    return checked_grid_left_to_right

---

#### Right-to-Left

In [76]:
def check_row_right_to_left(tree_grid_right_to_left):
    """Create a grid of trees visible for each row when viewed from right to left."""
    checked_grid_right_to_left = create_empty_checked_grid()
    
    # For each row viewed from right to left (reversed row)
    for i, row in enumerate(tree_grid_right_to_left):
        # Find the tallest tree for that row
        tallest_tree_height = max(row)
        
        # Iterate through each tree in the row
        for j, tree in enumerate(row):
            # For the first (when row is reversed, last if row is viewed left to right)
            # tree in the row, set it as the current tallest tree
            if j == 0:
                current_tree = row[j]
                checked_grid_right_to_left[i][j] = 1
                # If the first tree is the tallest tree, skip to checking the next row
                if current_tree == tallest_tree_height:
                    break
                
            # If the tree is not the first (last) tree in the row, and the tree is the same
            # height as the tallest tree
            elif tree == tallest_tree_height:
                # Add a visible tree marker (1) to the checked_trees grid at the tree's
                # coordinates
                checked_grid_right_to_left[i][j] = 1
                # Then skip to checking the next row
                break
            
            # If the tree is not the first (last) in the row, and it is not the tallest tree
            elif tree != tallest_tree_height:
                # If the tree's height is taller than the current tallest tree
                if tree > current_tree:
                    # Add a visible tree marker (1) to the checked_trees grid
                    checked_grid_right_to_left[i][j] = 1
                    # Then, change the current_tree to that tree, so that each following tree
                    # must be taller than it to be considered visible.
                    current_tree = tree
                    
                # The tree is not visible if it is not taller than the current tallest tree,
                # so continue checking the rest of the trees in the current row until the
                # tallest tree is found.
                elif tree <= current_tree:
                    continue
    
    # Reverse the checked grid so that the checked tree markers are in the correct coordinates
    # as viewed on the checked grid from left to right.
    
    checked_grid_right_to_left = np.fliplr(checked_grid_right_to_left)  
                
    return checked_grid_right_to_left

---

### Column Checks

#### Up to Down

In [99]:
def check_column_up_to_down(tree_grid_up_to_down):
    """Create a grid of trees visible for each column when viewed from up to down."""
    
    # Initialize empty grid to add visible trees (1s) to
    checked_grid_up_to_down = create_empty_checked_grid()
    
    # For each column in the tree grid
    for i, column in enumerate(tree_grid_up_to_down):
        # Find the tallest tree for that column
        tallest_tree_height = max(column)
        
        for j, tree in enumerate(column):
            # For the first tree in the column, set it as the current tallest tree
            if j == 0:
                current_tree = column[j]
                checked_grid_up_to_down[i][j] = 1
                # If the first tree is the tallest tree, we don't need to check the rest of
                # the trees in the column, we can skip to checking the next column
                if current_tree == tallest_tree_height:
                    break
            
            # If the tree is not the first in the column, and the tree is the same height
            # as the tallest tree
            elif tree == tallest_tree_height:
                # Add a visible tree marker (1) to the checked_trees grid
                checked_grid_up_to_down[i][j] = 1
                # Then skip to the next column
                break
            
            # If the tree is not the first in the column, and it is not the tallest tree
            elif tree != tallest_tree_height:
                # If the tree's height is taller than the current tallest tree
                if tree > current_tree:
                    # Add a visible tree marker (1) to the checked_trees grid
                    checked_grid_up_to_down[i][j] = 1
                    # Then, change the current_tree to that tree, so that each following tree
                    # must be taller than it to be considered visible.
                    current_tree = tree
                    
                # If the tree is not the first in the column, and the tree is not taller than
                # the current tallest tree, it is not visible, so continue checking the trees
                # in the current column until the tallest tree is found.
                elif tree <= current_tree:
                    continue
                
    # Flip the checked grid to reflect the trees visible from the original grid point of view
    checked_grid_up_to_down = np.fliplr(checked_grid_up_to_down)
    checked_grid_up_to_down = np.rot90(checked_grid_up_to_down, 1)
    
    return checked_grid_up_to_down

---

#### Down to Up

In [117]:
def check_column_down_to_up(tree_grid_down_to_up):
    """Create a grid of trees visible for each column when viewed from down to up."""
    
    # Initialize empty grid to add visible trees (1s) to
    checked_grid_down_to_up = create_empty_checked_grid()
    
    # For each column in the tree grid
    for i, column in enumerate(tree_grid_down_to_up):
        # Find the tallest tree for that column
        tallest_tree_height = max(column)
        
        for j, tree in enumerate(column):
            # For the first (last) tree in the column, set it as the current tallest tree
            if j == 0:
                current_tree = column[j]
                checked_grid_down_to_up[i][j] = 1
                # If the first tree is the tallest tree, we don't need to check the rest of
                # the trees in the column, we can skip to checking the next column
                if current_tree == tallest_tree_height:
                    break
            
            # If the tree is not the first (last) in the column, and the tree is the same
            # height as the tallest tree
            elif tree == tallest_tree_height:
                # Add a visible tree marker (1) to the checked_trees grid
                checked_grid_down_to_up[i][j] = 1
                # Then skip to the next column
                break
            
            # If the tree is not the first (last) in the column, and it is not the tallest
            # tree
            elif tree != tallest_tree_height:
                # If the tree's height is taller than the current tallest tree
                if tree > current_tree:
                    # Add a visible tree marker (1) to the checked_trees grid
                    checked_grid_down_to_up[i][j] = 1
                    # Then, change the current_tree to that tree, so that each following
                    # (preceding) tree must be taller than it to be considered visible.
                    current_tree = tree
                    
                # If the tree is not the first (last) in the column, and the tree is not
                # taller than the current tallest tree, it is not visible, so continue
                # checking the trees in the current column until the tallest tree is found.
                elif tree <= current_tree:
                    continue
                
    # Flip the checked grid to reflect the trees visible from the original grid point of view
    checked_grid_down_to_up = np.rot90(checked_grid_down_to_up, 1)
    
    return checked_grid_down_to_up

---

## Checked Grids

---

### Edges Check Grid

In [71]:
checked_grid_edges = add_edge_trees_to_checked_grid(tree_grid_original)
checked_grid_edges

array([[1, 1, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

---

### Row Check Left to Right Grid

In [72]:
checked_grid_left_to_right = check_row_left_to_right(tree_grid_left_to_right)
checked_grid_left_to_right

array([[1, 0, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1],
       [1, 1, 0, 1, 0]], dtype=int16)

---

### Row Check Right to Left Grid

In [77]:
checked_grid_right_to_left = check_row_right_to_left(tree_grid_right_to_left)
checked_grid_right_to_left

array([[0, 0, 0, 1, 1],
       [0, 0, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1]], dtype=int16)

---

### Column Check Up to Down Grid

In [100]:
checked_grid_up_to_down = check_column_up_to_down(tree_grid_up_to_down)
checked_grid_up_to_down

array([[1, 1, 1, 1, 1],
       [0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]], dtype=int16)

---

### Column Check Down to Up Grid

In [118]:
checked_grid_down_to_up = check_column_down_to_up(tree_grid_down_to_up)
checked_grid_down_to_up

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

---

## Merge All Checked Grids

In [128]:
merged_checked_grids = (checked_grid_edges|checked_grid_left_to_right \
                    |checked_grid_right_to_left|checked_grid_up_to_down \
                    |checked_grid_down_to_up)
merged_checked_grids

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]], dtype=int16)

---

## Count Visible Trees

In [129]:
np.count_nonzero(merged_checked_grids)

21

In [126]:
def count_visible_trees(merged_checked_grids):
    
    number_visible = np.count_nonzero(merged_checked_grids)
    